In [2]:
import openai
from dotenv import load_dotenv
import json
from datetime import datetime
import os 

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')

In [36]:
tools = [
    {
        "type": "function",  # Define the function attribute,
        "function": {
            "name": "extract_government_positions", 
            "description": "Extract government positions and their details from the provided text",
            "parameters": {
            "type": "object",
            "properties": {
                "town_name": {
                    "type": "string",
                    "description": "Name of the town."
                },
                "government_positions": {
                    "type": "array",
                    "description": "List of government positions and their details in the town.",
                    "items": {
                        "type": "object",
                        "properties": {
                            "position_title": {
                                "type": "string",
                                "description": "Title of the position."
                            },
                            "description": {
                                "type": "string",
                                "description": "Description of the position."
                            },
                            "next_election_date": {
                                "type": "string",
                                "format": "date",
                                "description": "Date of the next election. Only future dates are relevant."
                            },
                            "filing_window_start": {
                                "type": "string",
                                "format": "date",
                                "description": "Start date of the filing window. Only future dates are relevant."
                            },
                            "filing_window_end": {
                                "type": "string",
                                "format": "date",
                                "description": "End date of the filing window. Only future dates are relevant."
                            },
                            "name_of_district": {
                                "type": "string",
                                "description": "Name of the district."
                            },
                            "state_of_district": {
                                "type": "string",
                                "description": "State of the district."
                            },
                            "other_relevant_info": {
                                "type": "string",
                                "description": "Any other relevant information."
                            },
                            "vacancy_date": {
                                "type": "string",
                                "format": "date",
                                "description": "Date when the position will be vacant. Only future dates are relevant."
                            }
                        },
                        "required": ["position_title", "vacancy_date"]
                    }
                }
            },
            "required": ["town_name", "government_positions"],
            "additionalProperties": False
            }
        }
    }
]


In [37]:
with open("./../shared_data/unstructured_data/ballotpedia.org_List_of_current_mayors_of_the_top_100_cities_in_the_United_States__Fort_Wayne,_Indiana.txt_unstructured.txt", "r") as file:
    text_content = file.read()


In [38]:
# # Read the content of the text file
# with open('information.txt', 'r') as file:
#     text_content = file.read()

# Define the town name
from openai import OpenAI
client = OpenAI(api_key=openai_api_key)
town_name = "Fort Wayne, Indiana"

# Create the response stream
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role": "system",
            "content": (
                "You are an expert in extracting government positions and their details from text. "
                "Always use the provided tool to extract and structure data as required."
            ),
        },
        {
            "role": "user",
            "content": f"Tell me about the government positions in {town_name}. Here is the information: {text_content}",
        },
    ],
    tools=tools,
    tool_choice="required",
)



In [39]:
# Extract the function call arguments
function_call_arguments = response.choices[0].message.function_call.arguments

# Parse the JSON string into a Python dictionary
parsed_arguments = json.loads(function_call_arguments)

# Pretty-print the JSON data
pretty_json = json.dumps(parsed_arguments, indent=4)
print(pretty_json)

AttributeError: 'NoneType' object has no attribute 'arguments'

In [34]:
# Extract the function call arguments
function_call_arguments = response_stream.choices[0].message.function_call.arguments

# Parse the JSON string into a Python dictionary
parsed_arguments = json.loads(function_call_arguments)

# Pretty-print the JSON data
pretty_json = json.dumps(parsed_arguments, indent=4)
print(pretty_json)

{
    "town_name": "Fort Wayne, Indiana",
    "government_positions": [
        {
            "position_title": "Mayor of Fort Wayne",
            "description": "The Mayor of Fort Wayne is the head of the city government, responsible for overseeing city operations and representing the city at official functions.",
            "next_election_date": "2027",
            "vacancy_date": "April 23, 2024",
            "other_relevant_info": "Sharon Tucker is the current Mayor, representing the Democratic Party."
        },
        {
            "position_title": "City Council Member",
            "description": "Members of the City Council represent various districts of Fort Wayne and are responsible for creating legislation, approving budgets, and passing ordinances.",
            "next_election_date": "2027",
            "vacancy_date": "April 23, 2027",
            "filing_window_start": "May 2, 2023",
            "filing_window_end": "July 15, 2023",
            "other_relevant_info": "

In [ ]:
# Check if the conversation was too long for the context window
if response['choices'][0]['message']['finish_reason'] == "length":
    print("Error: The conversation was too long for the context window.")
    # handle_length_error(response)
    
# Check if the model's output included copyright material (or similar)
if response['choices'][0]['message']['finish_reason'] == "content_filter":
    print("Error: The content was filtered due to policy violations.")
    # handle_content_filter_error(response)
    
# Check if the model has made a tool_call. This is the case either if the "finish_reason" is "tool_calls" or if the "finish_reason" is "stop" and our API request had forced a function call
if response['choices'][0]['message']['finish_reason'] == "stop":
 
    
    
# Catch any other case, this is unexpected
else:
    print("Unexpected finish_reason:", response['choices'][0]['message']['finish_reason'])
    # Handle unexpected cases as needed
    # handle_unexpected_case(response)

In [26]:
# Function to check if the position is available
def is_position_available(vacancy_date_str):
    try:
        vacancy_date = datetime.strptime(vacancy_date_str, "%Y-%m-%d")
        return vacancy_date <= datetime.now()
    except ValueError:
        return "Invalid date format"

# Process the response stream
for response in response_stream:
    choice = response.choices[0]
    function_call = getattr(choice, 'function_call', None)
    if function_call and choice.finish_reason == 'function_call':
           if function_call.name == 'extract_government_positions':
               parameters = json.loads(function_call.arguments)
               town_name = parameters['town_name']
               government_positions = parameters['government_positions']
               print(f"Town: {town_name}")
               print("Government Positions:")
               for position in government_positions:
                print(f"- Position Title: {position['position_title']}")
                print(f"  Description: {position.get('description', 'N/A')}")
                print(f"  Next Election Date: {position.get('next_election_date', 'N/A')}")
                print(f"  Filing Window Start: {position.get('filing_window_start', 'N/A')}")
                print(f"  Filing Window End: {position.get('filing_window_end', 'N/A')}")
                print(f"  Name of District: {position.get('name_of_district', 'N/A')}")
                print(f"  State of District: {position.get('state_of_district', 'N/A')}")
                print(f"  Other Relevant Info: {position.get('other_relevant_info', 'N/A')}")
                vacancy_date = position['vacancy_date']
                print(f"  Vacancy Date: {vacancy_date}")
                print(f"  Is Available: {is_position_available(vacancy_date)}")
    else:
        print(choice.message.content)


AttributeError: 'tuple' object has no attribute 'choices'